In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import f1_score, accuracy_score, make_scorer
from sklearn.naive_bayes import MultinomialNB, CategoricalNB, GaussianNB, BernoulliNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from statistics import mean
from catboost import CatBoostClassifier
import eli5
from sklearn.pipeline import make_pipeline
from IPython import display

In [28]:
from sklearn.linear_model import LogisticRegressionCV
import scipy
import numpy as np
from lime.lime_text import LimeTextExplainer
from lime import lime_text

In [45]:
data = pd.read_parquet("train_no_trash.pqt")
class_map = pd.factorize(data['Класс документа'])
class_map = {document_class: document_class_index
            for document_class_index, document_class in zip(class_map[0], class_map[1])}

data['Класс документа (индекс)'] = data['Класс документа'].apply(class_map.get)
clf = make_pipeline(TfidfVectorizer(lowercase=False, analyzer='word', min_df=3), MultinomialNB())
clf.fit(data['Текст документа'], data['Класс документа (индекс)'])
#clf = CatBoostClassifier().load_model("CatboostBaseline.cbm")

short_vers = list(map(lambda x: str(x)[-15:], data['Класс документа'].unique()))
explainer = LimeTextExplainer(class_names=list(class_map.keys())) 
exp = explainer.explain_instance(data['Текст документа'].iloc[40], 
                                 clf.predict_proba, 
                                 num_features=30)
#exp.show_in_notebook()
exp.save_to_file("GGG")
#pd.DataFrame(exp.as_list(), columns=["Слово", "Уверенность"]).query("Уверенность > 0").sort_values(by='Уверенность', ascending=False)